In [19]:
import ipyvolume as ipv

In [20]:
import PIL.Image
import numpy as np

In [21]:
SKY_DIST = 7000  # distance to celestial sphere

In [37]:
def plot_earth_wire(ipv):
    nx = 40
    ny = 15
    lon = np.linspace(0, 2*np.pi, nx)
    lat = np.linspace(-np.pi/2, np.pi/2, ny)
    lon, lat = np.meshgrid(lon, lat)
    x = np.cos(lon)*np.cos(lat)
    y = np.sin(lon)*np.cos(lat)
    z = np.sin(lat)    
    ipv.plot_wireframe(
        x, y, z,
    )

def plot_earth(ipv):
    nx = 500
    ny = 250
    lon = np.linspace(0, 2*np.pi, nx)
    lat = np.linspace(-np.pi/2, np.pi/2, ny)
    lon, lat = np.meshgrid(lon, lat)
    x = np.cos(lon)*np.cos(lat)
    y = np.sin(lon)*np.cos(lat)
    z = np.sin(lat)    
    u = lon/(2*np.pi)  # map to 0->1
    u = np.roll(u, nx//2)  # +x = greenwich
    v = lat/np.pi-0.5
    img = PIL.Image.open('earth.jpg')
    ipv.plot_mesh(
        x, y, z,
        u=u, v=v,
        texture=img,
        wireframe=False
    )

In [38]:
def rubin_position():
    """Compute x/y/z coordinates of Rubin observatory
    in Earth-Centered Earth-Fixed frame
    """
    rubin_lat = -np.deg2rad(30+14/60+40.7/3600)
    rubin_lon = -np.deg2rad(70+44/60+57.9/3600)

    # Position of Rubin Observatory
    x = np.cos(rubin_lon)*np.cos(rubin_lat)
    y = np.sin(rubin_lon)*np.cos(rubin_lat)
    z = np.sin(rubin_lat)    
    return np.array([x, y, z])

In [39]:
def rubin_NEZ():
    """Compute Rubin observatory North-East-Zenith unit vectors in 
    Earth-Centered Earth-Fixed frame
    """
    rubin_lat = -np.deg2rad(30+14/60+40.7/3600)
    rubin_lon = -np.deg2rad(70+44/60+57.9/3600)

    # North
    # (= derivative of unit vectors wrt latitude)
    north = np.array([
        -np.cos(rubin_lon)*np.sin(rubin_lat),
        -np.sin(rubin_lon)*np.sin(rubin_lat),
        np.cos(rubin_lat)
    ])
    
    # Zenith
    # (parallel to Rubin position)
    zenith = rubin_position()

    # East = North x Up
    east = np.cross(north, zenith)

    # Note, unit vectors along _rows_
    return np.array([north, east, zenith])

In [40]:
def plot_rubin_position(ipv):
    x, y, z = rubin_position()
    ipv.scatter(
        np.array([x]),
        np.array([y]),
        np.array([z]),
        color='white', 
        size=1
    )    

In [41]:
def plot_rubin_NEZ(ipv):
    """Plot North, East, and Zenith triad.
    """
    
    north, east, zenith = rubin_NEZ()
    x, y, z = rubin_position()
    
    length = 3
    ipv.plot(
        np.array([x, x+north[0]*length]),
        np.array([y, y+north[1]*length]),
        np.array([z, z+north[2]*length]),
        color='blue'
    )
    ipv.plot(
        np.array([x, x+east[0]*length]),
        np.array([y, y+east[1]*length]),
        np.array([z, z+east[2]*length]),
        color='red'
    )
    ipv.plot(
        np.array([x, x+zenith[0]*length]),
        np.array([y, y+zenith[1]*length]),
        np.array([z, z+zenith[2]*length]),
        color='green'
    )    

In [42]:
def get_boresight(alt, az):
    # az = 0 means North, 90 means East
    # alt = 0 means horizon, 90 means zenith
    north, east, zenith = rubin_NEZ()
    boresight = zenith
    # Start at zenith
    # Rotate -(90 - alt) about East
    # Use Rodrigues' rotation formula https://en.wikipedia.org/wiki/Rodrigues'_rotation_formula
    # v_rot = v cos(th) + (k x v) sin(th) + k (k . v) (1 - cos(th))
    #   with v = boresight, k = east, th = -(90 - alt)
    # reduces to
    # v1 = zenith cos(th) - north sin(th) + 0
    th = -(np.pi/2 - alt)
    v1 = zenith * np.cos(th) - north * np.sin(th)
    # Now rotate result -az around zenith
    boresight = v1 * np.cos(-az) + np.cross(zenith, v1) * np.sin(-az) + zenith * np.dot(zenith, v1) * (1 - np.cos(-az))
    return boresight

In [43]:
def plot_boresight(ipv, alt, az):
    x, y, z = rubin_position()
    boresight = get_boresight(alt, az)
    length = SKY_DIST
    ipv.plot(
        np.array([x, x+boresight[0]*length]),
        np.array([y, y+boresight[1]*length]),
        np.array([z, z+boresight[2]*length]),
        color='magenta'
    )    

In [44]:
def plot_celestial_sphere(ipv):
    rx, ry, rz = rubin_position()
    dist = SKY_DIST
    th = np.linspace(0, 2*np.pi, 100)
    for alt in np.arange(-90, 91, 10):
        x = np.cos(th)*np.cos(np.deg2rad(alt))*dist
        y = np.sin(th)*np.cos(np.deg2rad(alt))*dist
        z = np.full_like(th, np.sin(np.deg2rad(alt)))*dist
        x += rx  # Center around Rubin observatory
        y += ry
        z += rz
        ipv.plot(x, y, z, color='white')
    for az in np.linspace(0, 2*np.pi, 24, endpoint=False):
        x = np.cos(az)*np.cos(th)*dist
        y = np.sin(az)*np.cos(th)*dist
        z = np.sin(th)*dist
        x += rx  # Center around Rubin observatory
        y += ry
        z += rz
        ipv.plot(x, y, z, color='white')

In [45]:
def sky_NEB(alt, az):
    """Compute sky North-East-Boresight unit vectors.
    """
    boresight = get_boresight(alt, az)
    # If we convert boresight to ECEF longitude, latitude,
    # Then North is again the derivative wrt latitude.
    lon = np.arctan2(boresight[1], boresight[0])
    lat = np.arcsin(boresight[2])  # assume boresight is normalized
    north = np.array([
        -np.cos(lon)*np.sin(lat),
        -np.sin(lon)*np.sin(lat),
        np.cos(lat)
    ])
    # East = North x boresight
    east = np.cross(north, boresight)
    return np.array([
        north,
        east,
        boresight
    ])

In [46]:
def plot_sky_NE(ipv, alt, az):
    x, y, z = rubin_position()
    north, east, boresight = sky_NEB(alt, az)
    x += boresight[0]*SKY_DIST
    y += boresight[1]*SKY_DIST
    z += boresight[2]*SKY_DIST
    
    length = 0.4*SKY_DIST
    ipv.plot(
        np.array([x, x+north[0]*length]),
        np.array([y, y+north[1]*length]),
        np.array([z, z+north[2]*length]),
        color='blue'
    )
    ipv.plot(
        np.array([x, x+east[0]*length]),
        np.array([y, y+east[1]*length]),
        np.array([z, z+east[2]*length]),
        color='red'
    )

In [47]:
def ten(s):
    ss = s.split()
    h = ss[0]
    m = ss[1]
    s = ss[2]
    return float(h) + float(m)/60 + float(s)/3600

def plot_constellations(ipv):
    # I think this is for Local Sidereal Time = 0h
    # Would need to rotate for any other LST
    if not hasattr(plot_constellations, 'data'):
        from astroquery.simbad import Simbad
        Simbad.add_votable_fields('typed_id')
        out = Simbad.query_objects(["HIP 67301", "HIP 65378"])
        HIPset = set()
        with open("constellationship.fab") as f:
            lines = f.readlines()
        for line in lines:
            HIPset.update([int(s) for s in line.split()[2:]])
        HIPlist = list(HIPset)
        data = Simbad.query_objects(
            [f"HIP {s}" for s in HIPlist]
        )
        data['HIPID'] = HIPlist
        plot_constellations.data = data
        plot_constellations.lines = lines
    data = plot_constellations.data
    lines = plot_constellations.lines
    for line in lines:
        endpoints = iter(line.split()[2:])        
        for first in endpoints:
            second = next(endpoints)
            firstrow = data[np.nonzero(data['HIPID'] == int(first))]
            secondrow = data[np.nonzero(data['HIPID'] == int(second))]
            ra0 = np.deg2rad(15*ten(firstrow['RA'][0]))
            dec0 = np.deg2rad(ten(firstrow['DEC'][0]))
            ra1 = np.deg2rad(15*ten(secondrow['RA'][0]))
            dec1 = np.deg2rad(ten(secondrow['DEC'][0]))
            x0 = np.cos(ra0)*np.cos(dec0)
            y0 = np.sin(ra0)*np.cos(dec0)
            z0 = np.sin(dec0)
            x1 = np.cos(ra1)*np.cos(dec1)
            y1 = np.sin(ra1)*np.cos(dec1)
            z1 = np.sin(dec1)
            
            x = np.array([x0, x1])*SKY_DIST
            y = np.array([y0, y1])*SKY_DIST
            z = np.array([z0, z1])*SKY_DIST
            ipv.plot(x, y, z, color='white')

In [48]:
# Input your desired boresight altitude and azimuth

# Away from the pole, North and East on the surface
# seem to usually be closely aligned with North and East on the sky.
alt = np.deg2rad(60)
az = np.deg2rad(90)  # East


# # Near the South Pole North and East on Earth's surface can be very different
# # than North and East on the sky
# alt = np.deg2rad(30)
# az = np.deg2rad(170)  # Southish

# Get a 3d visualization including
# Earth/Sky/Equatorial grid/Constellations
# Unit vectors for Earth surface North (blue)  East (red)  Zenith (green)
# Boresight of observation (magenta)
# Unit vectors for Sky North (blue)  East (red) 
ipv.figure(width=500, height=400)
# plot_earth_wire(ipv)
plot_earth(ipv)
plot_rubin_position(ipv)
plot_rubin_NEZ(ipv)
plot_boresight(ipv, alt, az)
plot_celestial_sphere(ipv)
plot_sky_NE(ipv, alt, az)
plot_constellations(ipv)

ipv.xyzlim(-2, 2)
ipv.style.box_off()
ipv.style.set_style_dark()
ipv.show()